In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("drill1Opportunity.csv",header=None)

In [3]:
data.rename(columns ={110: 'labels'}, inplace =True)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,labels
0,20,1036,108,-197,974,254,-258,1052,-34,88,...,358,-821,69,79,13,79,-69,13,168,1
1,0,1038,92,-212,952,229,-250,1066,-22,98,...,363,-821,52,58,9,58,-52,9,168,1
2,67,1064,71,-224,983,218,-162,1041,-80,113,...,359,-814,28,19,10,19,-28,10,168,1
3,48,1056,20,-207,983,196,-146,1004,-12,143,...,354,-810,12,50,-16,50,-12,-16,168,1
4,61,1058,9,-190,979,192,-257,962,-13,181,...,359,-820,-4,19,3,19,4,3,168,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54915 entries, 0 to 54914
Columns: 111 entries, 0 to labels
dtypes: int64(111)
memory usage: 46.5 MB


In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
encoder = OneHotEncoder()

In [7]:
one_hot_labels = encoder.fit_transform(data['labels'].values.reshape(-1,1)).toarray()
one_hot_labels.shape

(54915, 18)

In [8]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,labels
0,20,1036,108,-197,974,254,-258,1052,-34,88,...,358,-821,69,79,13,79,-69,13,168,1
1,0,1038,92,-212,952,229,-250,1066,-22,98,...,363,-821,52,58,9,58,-52,9,168,1
2,67,1064,71,-224,983,218,-162,1041,-80,113,...,359,-814,28,19,10,19,-28,10,168,1
3,48,1056,20,-207,983,196,-146,1004,-12,143,...,354,-810,12,50,-16,50,-12,-16,168,1
4,61,1058,9,-190,979,192,-257,962,-13,181,...,359,-820,-4,19,3,19,4,3,168,1


In [9]:
# split between train and test sets: 80% train, 20% test
data.drop(['labels'], 1, inplace=True)
train_size = int(len(data)/100*80)
train_data = data[0:train_size]
test_data = data[train_size:len(data)]
print(len(data))
print(train_data.shape)
print(test_data.shape)
train_labels = one_hot_labels[0:train_size]
test_labels = one_hot_labels[train_size:len(data)]
print(train_labels.shape)
print(test_labels.shape)
step_size = 24
sensors = 110;

54915
(43932, 110)
(10983, 110)
(43932, 18)
(10983, 18)


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [11]:
-

In [12]:
train_helper = trainerHelper(scaled_train_data, scaled_test_data, train_labels, test_labels, step_size, sensors)
new_train_helper = trainerHelper(scaled_train_data, scaled_test_data, train_labels, test_labels, step_size, sensors)

1830
1830


In [13]:
batch_test, y_test = train_helper.get_train_batch()
y_test.shape

(1, 18)

In [14]:
# NETWORK ARCHITECTURE
import tensorflow as tf
tf.reset_default_graph()

In [15]:
window_size = 24
num_sensor_data = 110
number_classes = 18
num_units = [window_size, window_size, window_size]

In [16]:
win_size = tf.constant(24, dtype=tf.float32)

In [17]:
# PLACEHOLDERS
X = tf.placeholder(tf.float32, shape=[None,window_size,num_sensor_data])
y = tf.placeholder(tf.float32, shape=[None, number_classes])
hold_prob = tf.placeholder(tf.float32);

In [18]:
def LSTM_with_Dropout(num_units, hold):
    _cell = tf.nn.rnn_cell.LSTMCell(num_units)
    _cell = tf.contrib.rnn.DropoutWrapper(_cell, output_keep_prob = hold)
    
    return _cell

In [19]:
fw_cell = [LSTM_with_Dropout(n, hold_prob) for n in num_units]
bw_cell = [LSTM_with_Dropout(n, hold_prob) for n in num_units]

In [20]:
bidi_output, _ , _ = tf.contrib.rnn.stack_bidirectional_rnn(fw_cell, bw_cell, tf.unstack(X,num_sensor_data,2), dtype=tf.float32)

In [21]:
fusion_output = tf.convert_to_tensor([tf.reduce_sum(t) / win_size for t in bidi_output])
fusion_output_1 = tf.reshape(fusion_output, shape=[-1, num_sensor_data])
print(fusion_output_1)

Tensor("Reshape:0", shape=(1, 110), dtype=float32)


In [22]:
dense_ouput = tf.contrib.layers.fully_connected(fusion_output_1, 18, activation_fn=None)

In [23]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=dense_ouput))

In [24]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [25]:
init = tf.global_variables_initializer()

In [27]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = train_helper.get_train_batch()
        
        sess.run(train,feed_dict={X:batch_x,y:batch_y,hold_prob:0.75})
        
        if i%100 == 0:
            print('Training is on step {}'.format(i))
    
    print('Training finished')
    print('Evaluating on training')
    test_steps = 500
    count = 0.0
    den = 0.0
    for m in range(test_steps):
        den = den + 1.0
        batch_x , batch_y = new_train_helper.get_train_batch() 
        output = dense_ouput.eval(feed_dict={X:batch_x,y:batch_y,hold_prob:1.0})
        match = np.argmax(output) == np.argmax(batch_y)
        match = match.astype(float)
        count = count + match
        
        if m%100 == 0:
            print('Testing is on step {}'.format(m))
            print('Partial Accuracy')
            print(count / den)
    
    print('Testing finished')

Training is on step 0
Training is on step 100
Training is on step 200
Training is on step 300
Training is on step 400
Training is on step 500
Training is on step 600
Training is on step 700
Training is on step 800
Training is on step 900
Training is on step 1000
Training is on step 1100
Training is on step 1200
Training is on step 1300
Training is on step 1400
Training is on step 1500
Training is on step 1600
Training is on step 1700
Training is on step 1800
Training is on step 1900
Training is on step 2000
Training is on step 2100
Training is on step 2200
Training is on step 2300
Training is on step 2400
Training is on step 2500
Training is on step 2600
Training is on step 2700
Training is on step 2800
Training is on step 2900
Training is on step 3000
Training is on step 3100
Training is on step 3200
Training is on step 3300
Training is on step 3400
Training is on step 3500
Training is on step 3600
Training is on step 3700
Training is on step 3800
Training is on step 3900
Training is 